In [7]:
import math
import numpy as np

LENGTH=20
WIDTH=1920
HEIGHT=1080
FRAMERATE=30
color_purple = (255,0,255)
color_white = (255,255,255)

In [8]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

import cv2
import numpy as np

class TrackedVehicle:
    def __init__(self, label, speed,time):
        self.label = label
        self.speed = speed
        self.time = time

class TrackableVehicle:
    def __init__(self, id: int,
                  entered_timestamp: float | None,
                  last_anchor, label, indim) -> None:
        self.id = id
        self.enter_timestamp = entered_timestamp
        self.last_anchor = list()
        self.last_anchor.append((int(last_anchor[0]),int(last_anchor[1])))
        self.labels = dict()
        self.labels[label] = 1
        self.indims = indim


    def label(self):
        return max(self.labels, key=self.labels.get)


    def finish(self, time):
        return TrackedVehicle(self.label(),LENGTH/(time-self.enter_timestamp),time-self.enter_timestamp if self.enter_timestamp is not None else 0)
    
    def update(self, label, anchor, isins):
        if label not in self.labels.keys():
            self.labels[label] = 1
        else:
            self.labels[label] +=1
        self.last_anchor.append((int(x), int(y+h/2)))  # x, y center point
        if len(self.last_anchor) > 10:
            self.last_anchor.pop(0)
        self.indim=isins

    def old_anchor(self):
        return self.last_anchor[0]

def avspeed(tracked):
    n = 0
    sp = 0
    for v in tracked:
        if v.time>=0.5: 
            sp+=v.speed
            n+=1
    if n==0: return "NaN"
    return (sp/n)*3.6

def check_in(x, y, poly):
    return Polygon(poly).contains(Point(x,y))

def in_any(x, y, polys):
    for poly in polys:
        if Polygon(poly).contains(Point(x,y)): return True
    return False

In [9]:
from collections import defaultdict

from ultralytics import YOLO

import json


video_path=r"videos/dataset/Test_Newest_2/000.mp4"
markup=r"videos/dataset/Test_markup/000.json"

with open(markup, 'r') as file:
    data = json.load(file)

zones = data['zones']



# Load the YOLOv8 model
model = YOLO('yolov8n.pt')

# Open the video file
cap = cv2.VideoCapture(video_path)

WIDTH=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
HEIGHT=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
FRAMERATE=cap.get(cv2.CAP_PROP_FPS)

dims = []
for i, arr in enumerate(zones):
    dims.append([])
    for j, arr2 in enumerate(arr):
        dims[i].append((int(WIDTH*arr2[0]),int(HEIGHT*arr2[1])))

areas = data['areas']

bdims = []
for i, arr in enumerate(areas):
    bdims.append([])
    for j, arr2 in enumerate(arr):
        bdims[i].append((int(WIDTH*arr2[0]),int(HEIGHT*arr2[1])))

# Store the track history
track_history = defaultdict(lambda: [])

needed_classes=[1,2,3,5,7]

all_ids=dict()
tracking=dict()
tracked=dict()

old_ids = []

font = cv2.FONT_HERSHEY_DUPLEX
color = (255,255,255)
fontsize = 1

count=0
# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    success, frame = cap.read()
    if success:
        # Run YOLOv8 tracking on the frame, persisting tracks between frames
        results = model.track(frame, persist=True, tracker='bytetrack.yaml', classes=needed_classes)

        # Get the boxes and track IDs
        tracks = results[0].boxes.cpu()
        boxes = results[0].boxes.xywh.cpu()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        # Visualize the results on the frame
        annotated_frame = results[0].plot()

        # Plot the tracks
        for box, track_id in zip(boxes, track_ids):
            x, y, w, h = box
            track = track_history[track_id]
            track.append((float(x), float(y+h/2)))  # x, y center point
            if len(track) > 90:  # retain 90 tracks for 90 frames
                track.pop(0)

            # Draw the tracking lines
            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(annotated_frame, [points], isClosed=False, color=(230, 230, 230), thickness=10)

        # Plot markup
        for i in range(len(dims)):
            cv2.line(annotated_frame, tuple(dims[i][0]), tuple(dims[i][1]), color_purple, thickness=4, lineType=8, shift=0)
            cv2.line(annotated_frame, tuple(dims[i][1]), tuple(dims[i][2]), color_purple, thickness=4, lineType=8, shift=0)
            cv2.line(annotated_frame, tuple(dims[i][2]), tuple(dims[i][3]), color_purple, thickness=4, lineType=8, shift=0)
            cv2.line(annotated_frame, tuple(dims[i][3]), tuple(dims[i][0]), color_purple, thickness=4, lineType=8, shift=0)

        for i in range(len(bdims)):
            cv2.line(annotated_frame, tuple(bdims[i][0]), tuple(bdims[i][1]), color_purple, thickness=4, lineType=8, shift=0)
            cv2.line(annotated_frame, tuple(bdims[i][1]), tuple(bdims[i][2]), color_purple, thickness=4, lineType=8, shift=0)
            cv2.line(annotated_frame, tuple(bdims[i][2]), tuple(bdims[i][3]), color_purple, thickness=4, lineType=8, shift=0)
            cv2.line(annotated_frame, tuple(bdims[i][3]), tuple(bdims[i][0]), color_purple, thickness=4, lineType=8, shift=0)
        
        
        for box, id, track in zip(boxes, track_ids, tracks):
            x,y,w,h = box
            isin = in_any(float(x), float(y+h/2), bdims)
            isins = in_any(float(x), float(y+h/2), dims)
            if isin: 
                cv2.circle(annotated_frame, tuple((int(x), int(y+h/2))), 10, (255,0,0), 10)
            if ((id not in tracking.keys()) and isin and isins) or (id in tracking.keys() and (not isin)):


                if id not in tracking.keys(): tracking[id] = TrackableVehicle(id,None,(int(x), int(y+h/2)),track.cls,isins)
                
                if isin and isins:
                    tracking[id].enter_timestamp = (1/FRAMERATE)*count

                elif (not isin) and tracking[id].indims != isins:
                    if (1/FRAMERATE)*count!=tracking[id].enter_timestamp:
                        tracked[tracking[id].label()] = tracking[id].finish((1/FRAMERATE)*count)
                        tracking.pop(id)
                
            if id in tracking.keys(): tracking[id].update(track.cls, (int(x), int(y+h/2)), isins)
        cv2.putText(annotated_frame,  f"tracking inside: {len(tracking)}", (100,100), font, fontsize, color=color)
        cv2.putText(annotated_frame, f"tracking inside: {len(tracking)}", (100,100), font, fontsize, color=color)
        cv2.putText(annotated_frame, f"tracking everywhere: {len(track_ids)}", (100,140), font, fontsize, color=color)
        cv2.putText(annotated_frame, f"tracked and counted: {len(tracked)}", (100,180), font, fontsize, color=color)
        cv2.putText(annotated_frame, f"average speed: {avspeed(tracked.values())} km/h", (100,220), font, fontsize, color=color)

        # Display the annotated frame
        cv2.imshow("MIET hack team", annotated_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        # Break the loop if the end of the video is reached
        break

    count+=1

# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()


0: 384x640 2 cars, 38.8ms
Speed: 10.3ms preprocess, 38.8ms inference, 14.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 22.8ms
Speed: 5.0ms preprocess, 22.8ms inference, 8.6ms postprocess per image at shape (1, 3, 384, 640)



QObject::moveToThread: Current thread (0x1465fa20) is not the object's thread (0x27ce6850).
Cannot move to target thread (0x1465fa20)

QObject::moveToThread: Current thread (0x1465fa20) is not the object's thread (0x27ce6850).
Cannot move to target thread (0x1465fa20)

QObject::moveToThread: Current thread (0x1465fa20) is not the object's thread (0x27ce6850).
Cannot move to target thread (0x1465fa20)

QObject::moveToThread: Current thread (0x1465fa20) is not the object's thread (0x27ce6850).
Cannot move to target thread (0x1465fa20)

QObject::moveToThread: Current thread (0x1465fa20) is not the object's thread (0x27ce6850).
Cannot move to target thread (0x1465fa20)

QObject::moveToThread: Current thread (0x1465fa20) is not the object's thread (0x27ce6850).
Cannot move to target thread (0x1465fa20)

QObject::moveToThread: Current thread (0x1465fa20) is not the object's thread (0x27ce6850).
Cannot move to target thread (0x1465fa20)

QObject::moveToThread: Current thread (0x1465fa20) is n

0: 384x640 2 cars, 43.6ms
Speed: 7.1ms preprocess, 43.6ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 26.6ms
Speed: 5.8ms preprocess, 26.6ms inference, 6.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 20.5ms
Speed: 2.8ms preprocess, 20.5ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 35.5ms
Speed: 6.1ms preprocess, 35.5ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 36.6ms
Speed: 3.0ms preprocess, 36.6ms inference, 7.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 36.7ms
Speed: 25.9ms preprocess, 36.7ms inference, 22.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 29.1ms
Speed: 5.7ms preprocess, 29.1ms inference, 8.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 36.8ms
Speed: 6.2ms preprocess, 36.8ms inference, 11.9ms postprocess per image at shape (1, 3, 384, 640)

0: 38

AttributeError: 'NoneType' object has no attribute 'int'

In [ ]:
model.model.names

NameError: name 'model' is not defined